# Benefit Cost Ratio

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd

## Import data

In [3]:
data_all_wide = pd.read_csv("Data/preprocessed_scenario_data.csv")

# Transform wide table to long table
data_all = (
    data_all_wide[
        (data_all_wide["Region"] == "World")
        & (data_all_wide["SLR adaptation"] != "without")
    ].set_index(
        [
            "Model",
            "Scenario",
            "Region",
            "Damage quantile",
            "SLR quantile",
            "SLR adaptation",
            "Target",
            "Discounting",
            "Variable",
            "Unit",
        ]
    )
    .rename_axis("Year", axis=1)
    .stack()
    .to_frame("Value")
    .reset_index()
)
data_all["Discounting"] = data_all["Discounting"].fillna("not_applicable")


In [4]:
## Prepare MIMOSA data

data_mimosa = data_all[data_all["Model"] == "MIMOSA"].copy()
data_mimosa["Scenario"] = ""


# The with-damage baseline scenario is the same for each discount rate, so copy it for all
_data_mimosa_baseline_with_damage = data_mimosa[
    (data_mimosa["Target"] == "baseline")
    & (data_mimosa["Damage quantile"] != "nodamages")
]
extra1_mimosa = pd.concat(
    [
        _data_mimosa_baseline_with_damage.replace({"not_applicable": discounting})
        for discounting in ["low", "medium", "high"]
    ]
)

# The nodamage baseline scenario is the same for each discount rate and damage quantile, so copy it for all
_data_mimosa_baseline_nodamages = data_mimosa[
    (data_mimosa["Target"] == "baseline")
    & (data_mimosa["Damage quantile"] == "nodamages")
]
extra2_mimosa = pd.concat(
    [
        _data_mimosa_baseline_nodamages.replace(
            {"not_applicable": discounting, "nodamages": damage_q}
        )
        for discounting in ["low", "medium", "high"]
        for damage_q in ["5", "50", "95"]
    ]
)
extra2_mimosa["Target"] = "nodam"

data_mimosa = pd.concat(
    [
        data_mimosa[
            (data_mimosa["Discounting"] != "not_applicable")
            & (data_mimosa["Damage quantile"] != "nodamages")
        ],
        extra1_mimosa,
        extra2_mimosa,
    ]
)
data_mimosa["SLR adaptation"] = "with"
data_mimosa["Unit"] = ""


In [5]:
## Prepare REMIND data

data_remind = data_all[
    (data_all["Model"] == "REMIND") & (data_all["Target"] != "rcp26")
].copy()

# In REMIND, RCP6.0 is baseline ( = no policy)
data_remind = data_remind.replace({"rcp60": "baseline"})


# The nodamage baseline scenario will be compared with all damage quantiles equally, so copy it three times and change target to "nodam"

_nodam_baseline_remind = data_remind[
    (data_remind["Damage quantile"] == "nodamages")
    & (data_remind["Target"] == "baseline")
]
extra1_remind = pd.concat(
    [
        _nodam_baseline_remind.replace({"nodamages": damage_q})
        for damage_q in ["5", "50", "95"]
    ]
)
extra1_remind["Target"] = "nodam"
extra1_remind["SLR adaptation"] = "with"

data_remind = pd.concat(
    [data_remind[data_remind["Damage quantile"] != "nodamages"], extra1_remind,]
)


In [6]:
## Prepare WITCH data

data_witch = data_all[data_all["Model"] == "WITCH"].copy()

# with-damage baselines are called "tax0" in WITCH, rename here:
data_witch = data_witch.replace({"tax0": "baseline"})

# "baudam" scenarios (baselines with damages, different from tax0 runs) are not necessary:
data_witch = data_witch[data_witch["Target"] != "baudam"]

# The nodamage baseline scenario will be compared with all damage quantiles equally, so copy it three times and change target to "nodam"

_nodam_baseline_witch = data_witch[
    (data_witch["Damage quantile"] == "nodamages")
    & (data_witch["Target"] == "baseline")
]
extra_witch = pd.concat(
    [
        _nodam_baseline_witch.replace({"nodamages": damage_q})
        for damage_q in ["5", "50", "95"]
    ]
)

extra_witch["Target"] = "nodam"
extra_witch["SLR adaptation"] = "with"

data_witch = pd.concat(
    [data_witch[data_witch["Damage quantile"] != "nodamages"], extra_witch]
)

### Util functions
Functions like transforming relative quantities to absolute quantities and calculating NPVs

In [7]:
INDEX_COLUMNS = ['Model', 'Region', 'Damage quantile', 'SLR adaptation', 'Target', 'Discounting', 'Variable', 'Year']
INDEX_COLUMNS_NO_VAR = [c for c in INDEX_COLUMNS if c != 'Variable']
INDEX_COLUMNS_NO_VAR_NO_TARGET = [c for c in INDEX_COLUMNS_NO_VAR if c != 'Target']

# 1. Calculate absolute damage and mitigation costs
def calc_absolute(data, variable='Damage Cost|%'):
    selection = data[data['Variable'].isin([variable, 'GDP|PPP'])]
    unstacked = selection.set_index(
        INDEX_COLUMNS
    )['Value'].unstack('Variable')

    absolute_values = (
        unstacked[variable] * unstacked['GDP|PPP']
    ).rename('Value').reset_index()

    return absolute_values.set_index(INDEX_COLUMNS_NO_VAR)['Value']

def calc_avoided_damages(data, baseline_target='baseline', mitig_target='cba'):

    data = data.copy().reset_index()
    # Check that there is only one variable
    if 'Variable' in data.columns and len(data['Variable'].unique()) > 1:
        raise Exception("Variable is not unique")

    selection = data[data['Target'].isin([baseline_target, mitig_target])]

    stacked = selection.set_index(INDEX_COLUMNS_NO_VAR)['Value'].unstack('Target')
    avoided_damages = stacked[baseline_target] - stacked[mitig_target]

    to_df = avoided_damages.rename('Value').reset_index()
    to_df['Target'] = mitig_target

    return to_df.set_index(INDEX_COLUMNS_NO_VAR)['Value']

def calc_npv(data, prtp, elasmu=1):

    sdr = prtp + elasmu * 0.021

    data = data.copy().reset_index()
    group_columns = [c for c in data.columns if c not in ['Year', 'Value']]

    grouped = data.groupby(group_columns)
    
    def npv(subdf):
        years = subdf['Year'].astype(float)
        discounted_values = np.exp(-sdr * (years - years.iloc[0])) * subdf['Value']
        return np.trapz(discounted_values, years)

    to_npv = grouped.apply(npv)

    return to_npv

def calc_bcr(avoided_damages, costs, prtp, select_only_discounting=None):
    npv_benefits = calc_npv(avoided_damages, prtp)
    npv_costs = calc_npv(costs, prtp)

    bcr = npv_benefits / npv_costs

    bcr_df = bcr.rename('Value').reset_index()
    bcr_df['PRTP'] = f'{prtp:.1%}/yr'

    if select_only_discounting is not None:
        bcr_df = bcr_df[bcr_df['Discounting'] == select_only_discounting]
    return bcr_df

## Calculations of BCR

In [8]:
def calc_bcr_full(data, model):
    gdp = data[(data['Variable'] == 'GDP|PPP')].set_index(INDEX_COLUMNS_NO_VAR)['Value']
    gdp_unstacked = gdp.unstack('Target')
    gdploss_cba = gdp_unstacked['nodam'] - gdp_unstacked['cba']
    gdploss_baseline = gdp_unstacked['nodam'] - gdp_unstacked['baseline']

    baseline_damages = gdploss_baseline

    cba_directdamages = calc_absolute(data[data['Target'] == 'cba'], variable='Damage Cost|%').droplevel('Target')
    cba_directpolicy = calc_absolute(data[data['Target'] == 'cba'], variable='Policy Cost|%').droplevel('Target')


    if model == "MIMOSA":
        # Calculate indirect upscaling factor to distribute indirect costs
        # between damages and policy costs
        indirect_scale_factor = gdploss_cba / (cba_directpolicy + cba_directdamages)
        indirect_scale_factor.loc[pd.IndexSlice[:,:,:,:,:,'2020']] = 1.0

        cba_damages = cba_directdamages * indirect_scale_factor
        cba_policy = cba_directpolicy * indirect_scale_factor
    
    else:
        # For WITCH and REMIND, the policy costs already include indirect costs
        cba_damages = gdploss_cba - cba_directpolicy
        cba_policy = cba_directpolicy

    costs = cba_policy.rename('Value')
    avoided_damages = (baseline_damages - cba_damages).rename('Value')

    bcr = pd.concat([
        calc_bcr(avoided_damages, costs, prtp, discounting)
        for discounting, prtp in zip(['low', 'medium', 'high'], [0.001, 0.015, 0.03])
    ])

    relative_costs = (costs / gdp).reset_index()
    relative_avoided_damages = (avoided_damages / gdp).reset_index()
    
    return bcr, relative_costs, relative_avoided_damages

In [9]:
bcr_mimosa, relative_costs_mimosa, relative_avoided_damages_mimosa = calc_bcr_full(data_mimosa, "MIMOSA")
bcr_remind, relative_costs_remind, relative_avoided_damages_remind = calc_bcr_full(data_remind, "REMIND")
bcr_witch, relative_costs_witch, relative_avoided_damages_witch = calc_bcr_full(data_witch, "WITCH")

bcr_values = pd.concat([bcr_mimosa, bcr_remind, bcr_witch])
relative_costs_values = pd.concat([relative_costs_mimosa, relative_costs_remind, relative_costs_witch])
relative_avoided_damages_values = pd.concat([relative_avoided_damages_mimosa, relative_avoided_damages_remind, relative_avoided_damages_witch])

bcr_values.to_csv("Data/output_benefit_cost_ratios.csv", index=False)

bcr_values

,Model,Region,Damage quantile,SLR adaptation,Discounting,Value,PRTP
1,MIMOSA,World,5,with,low,0.528401,0.1%/yr
4,MIMOSA,World,50,with,low,3.155090,0.1%/yr
7,MIMOSA,World,95,with,low,3.538762,0.1%/yr
2,MIMOSA,World,5,with,medium,0.270417,1.5%/yr
5,MIMOSA,World,50,with,medium,2.704602,1.5%/yr
8,MIMOSA,World,95,with,medium,2.964245,1.5%/yr
0,MIMOSA,World,5,with,high,-0.081209,3.0%/yr
3,MIMOSA,World,50,with,high,2.390659,3.0%/yr
6,MIMOSA,World,95,with,high,2.603852,3.0%/yr
1,REMIND,World,5,with,medium,0.957992,1.5%/yr


### Plot

In [10]:
relative_avoided_damages_values['Variable'] = 'Avoided Damages|% GDP'
relative_avoided_damages_values = relative_avoided_damages_values[relative_avoided_damages_values['Target'] == 'cba']

relative_costs_values['Variable'] = 'Policy Cost|% GDP'
relative_costs_values = relative_costs_values[relative_costs_values['Target'] == 'cba']

temperatures = pd.concat([data_mimosa, data_remind, data_witch])
temperatures = temperatures[
    (temperatures['Variable'] == 'Temperature|Global Mean')
    & temperatures['Target'].isin(['cba', 'baseline'])
    & (temperatures['SLR adaptation'] == 'with')
].astype({'Year': int})

costs_and_benefits = pd.concat([
    relative_avoided_damages_values,
    relative_costs_values,
]).astype({'Year': int})

costs_and_benefits.to_csv("Data/output_costs_and_benefits_over_time.csv", index=False)

bcr_values['Model'] = pd.Categorical(bcr_values['Model'], ['MIMOSA', 'WITCH', 'REMIND'], True)

In [11]:
from utils import plot_bcr
fig_with_temp = plot_bcr.fig_bcr(
    relative_avoided_damages_values, relative_costs_values, temperatures, bcr_values
)
fig_with_temp

In [12]:
# fig_with_temp.write_image("Figures/figure5_cba_benefit_cost_ratios_3models.png", scale=3)

In [13]:
fig = plot_bcr.fig_bcr(
    relative_avoided_damages_values, relative_costs_values, None, bcr_values
)
fig

In [14]:
fig.write_image("Figures/figure5_cba_benefit_cost_ratios.png", scale=3)

In [15]:
fig_p95 = plot_bcr.fig_bcr(
    relative_avoided_damages_values, relative_costs_values, None, bcr_values, damage_quantile="95"
)
fig_p95

In [16]:
fig_p95.write_image("Figures/SI_figure_cba_benefit_cost_ratios_p95.png", scale=3)